In [ ]:
import pyspark
from pyspark.sql import SparkSession

Understanding how spark works:
- groupby
- joins
- RDD
- mapPartitions
- [deploy spark locally](https://spark.apache.org/docs/latest/spark-standalone.html)

### Creating spark server

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()
# http://localhost:4040/jobs/

### Getting the data

In the terminal, run: `wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-01.parquet`


In [ ]:
FILE_PATH = "../fhvhv_tripdata_2024-01.parquet"

In [ ]:
df = spark.read.parquet(FILE_PATH)

In [ ]:
df.head(3)

In [ ]:
df.schema

In [ ]:
df.printSchema()

### Partitionning

In [ ]:
df = df.repartition(24)

In [ ]:
# df.write.parquet('fhvhv/2024/01')

### Select

In [ ]:
df.select('trip_time', 'trip_miles', 'base_passenger_fare', 'driver_pay', 'pickup_datetime', 'dropoff_datetime') \
    .filter(df.hvfhs_license_num == 'HV0003') \
    .show()

### Register Table

In [ ]:
df.createOrReplaceTempView('yellow')

In [ ]:
# 
df_result = spark.sql("""
SELECT
    sum(base_passenger_fare) as total_passenger_fare,
    sum(trip_miles) as total_trip_miles, 
    sum(trip_time) as total_trip_time
FROM 
    yellow
WHERE
    pickup_datetime between '2024-01-01 00:00:00' and '2024-02-01 00:00:00'
""")

In [ ]:
df_result.show()